DISCLAIMER: 

This Python notebook uses python 3.12.0. Other versions may have errors.

In [2]:
%pip install fastapi uvicorn nest_asyncio

  Obtaining dependency information for uvicorn from https://files.pythonhosted.org/packages/b1/4b/4cef6ce21a2aaca9d852a6e84ef4f135d99fcd74fa75105e2fc0c8308acd/uvicorn-0.34.2-py3-none-any.whl.metadata
  Obtaining dependency information for click>=7.0 from https://files.pythonhosted.org/packages/7e/d4/7ebdbd03970677812aac39c869717059dbb71a4cfc033ca6e5221787892c/click-8.1.8-py3-none-any.whl.metadata
  Obtaining dependency information for h11>=0.8 from https://files.pythonhosted.org/packages/04/4b/29cac41a4d98d144bf5f6d33995617b185d14b22401f75ca86f384e87ff1/h11-0.16.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/62.5 kB ? eta -:--:--
   ------ --------------------------------- 10.2/62.5 kB ? eta -:--:--
   ------------------- -------------------- 30.7/62.5 kB 435.7 kB/s eta 0:00:01
   ---------------------------------------- 62.5/62.5 kB 669.8 kB/s eta 0:00:00
   ---------------------------------------- 0.0/98.2 kB ? eta -:--:--
   -----------------------------


[notice] A new release of pip is available: 23.2.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from fastapi import FastAPI
import nest_asyncio
import uvicorn  

nest_asyncio.apply()

app = FastAPI()

@app.get("/")
def read_root():
    return {"Name" : "Luke"}

uvicorn.run(app, host="127.0.0.1", port=8000)

INFO:     Started server process [39424]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:57674 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:57674 - "GET /favicon.ico HTTP/1.1" 404 Not Found


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [39424]


Slightly more advanced 

In [ ]:
%pip install fastapi uvicorn nest_asyncio python-multipart pandas

In [18]:
from fastapi import FastAPI, File, UploadFile
from fastapi.responses import JSONResponse
import pandas as pd
import nest_asyncio
import uvicorn  
import numpy as np
from io import StringIO

nest_asyncio.apply()

data_records = []

app = FastAPI()

@app.on_event("startup")
def load_data():
    global data_records
    df = pd.read_csv("../MLModel/Data/Shapes.csv")
    data_records = df.to_dict(orient="records")

@app.get("/get-data")
async def gather_data():
    return JSONResponse(content=data_records)

uvicorn.run(app, host="127.0.0.1", port=8000)

C:\Users\12489\AppData\Local\Temp\ipykernel_39424\2447679922.py:15: DeprecationWarning: 
        on_event is deprecated, use lifespan event handlers instead.

        Read more about it in the
        [FastAPI docs for Lifespan Events](https://fastapi.tiangolo.com/advanced/events/).
        
  @app.on_event("startup")
INFO:     Started server process [39424]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:60589 - "GET /get-data HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [39424]


Lets go slightly further...

In [ ]:
%pip install fastapi uvicorn pandas scikit-learn pydantic

In [21]:
from fastapi import FastAPI
from fastapi.responses import JSONResponse
from pydantic import BaseModel
import pandas as pd
import nest_asyncio
import uvicorn  
import numpy as np
from sklearn.tree import DecisionTreeClassifier

df = pd.DataFrame()

app = FastAPI()

model = None
feature_columns = ["Edges","Length","Width"]

class ShapeInput(BaseModel):
    edges: int
    length: int
    height: int

def load_data():
    global df
    df = pd.read_csv("../MLModel/Data/Shapes.csv")
    

def train_model():
    global model
    global df

    X = df[feature_columns]
    y = df["Shape"]

    model = DecisionTreeClassifier()
    model.fit(X, y)

@app.on_event("startup")
def startup():
    load_data()
    train_model()

@app.get("/get-data")
async def gather_data():
    global df
    data_records = df.to_dict(orient="records")
    return JSONResponse(content=data_records)

@app.post("/predict")
def predict_shape(input: ShapeInput):
    input_array = np.array([[input.edges, input.length, input.height]])
    prediction = model.predict(input_array)
    return {"predicted_shape": prediction[0]}

uvicorn.run(app, host="127.0.0.1", port=8000)

C:\Users\12489\AppData\Local\Temp\ipykernel_39424\3619225509.py:37: DeprecationWarning: 
        on_event is deprecated, use lifespan event handlers instead.

        Read more about it in the
        [FastAPI docs for Lifespan Events](https://fastapi.tiangolo.com/advanced/events/).
        
  @app.on_event("startup")
Task exception was never retrieved
future: <Task finished name='Task-106' coro=<Server.serve() done, defined at c:\Users\12489\AppData\Local\Programs\Python\Python312\Lib\site-packages\uvicorn\server.py:68> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "c:\Users\12489\AppData\Local\Programs\Python\Python312\Lib\site-packages\uvicorn\main.py", line 580, in run
    server.run()
  File "c:\Users\12489\AppData\Local\Programs\Python\Python312\Lib\site-packages\uvicorn\server.py", line 66, in run
    return asyncio.run(self.serve(sockets=sockets))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\12489\AppData\Roaming\Python\Python3

INFO:     127.0.0.1:62820 - "POST /predict HTTP/1.1" 200 OK


c:\Users\12489\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [39424]
